In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
from zipfile import ZipFile
from sklearn.model_selection import train_test_split

PNS_DATA = Path('../data/raw/PNS_2019/PNS_2019.zip').resolve()
zip_pense = ZipFile(PNS_DATA)

path_unzip = zip_pense.namelist()[0]
df = pd.read_csv(zip_pense.open(path_unzip), delimiter=';', low_memory=False)

In [2]:
print("O dataset PNS possui {} linhas e {} colunas (features).".format(*df.shape))

O dataset PNS possui 293726 linhas e 1087 colunas (features).


Neste projeto estamos interessados nas variáveis relacionadas a indicativo de que o entrevistado possui depressão, vamos usar a questão Q092 (Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?) como referência de filtragem de dados, questões mais específicas como 'Por causa da depressão Toma medicamentos', que também poderiam ser utilizadas como parâmetro de indicativo de depressão são, na verdade, dependentes da variável Q092.

In [3]:
selected = df[df['Q092'].isin([1, 2])] # pega somente os registros que foram respondidos como sim ou não
print("A quantidade de dados investigável dos dados são {}, {:02f}% do total.".format(selected.shape[0], 100*selected.shape[0]/df.shape[0]))

A quantidade de dados investigável dos dados são 90846, 30.928825% do total.


In [4]:
print("Numero de casas entrevistadas: ", selected.groupby(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS']).size().reset_index().shape[0])
print("Numero de pessoas entrevistadas: ", selected.groupby(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS', 'C00301']).size().reset_index().shape[0])

Numero de casas entrevistadas:  90846
Numero de pessoas entrevistadas:  90846


In [44]:
selected['M00302'].value_counts() # Entrevista do adulto selecionado: 1 - realizada

M00302
1.0    1107
2.0     570
Name: count, dtype: int64

In [45]:
selected['Q092'].value_counts()

Q092
2.0    82514
1.0     8332
Name: count, dtype: int64

In [34]:
selected['V0024']

0         1110011
9         1110011
10        1110011
18        1110011
19        1110011
           ...   
293709    5310220
293712    5310220
293713    5310220
293718    5310220
293720    5310220
Name: V0024, Length: 90846, dtype: int64

#### Filtragem das colunas de interesse

In [ ]:
PNS_SELECTED_VAR = {
    # Parte 1 - Identificação e Controle
    'V0001': "Unidade da Federação",
    'V0022': "Total de moradores",
    'V0026': "Tipo de situação censitária",
    'V0031': "Tipo de área",
    'V0025A': "Seleção do morador de 15 anos ou mais para responder o questionário individual",

    # VARIÁVEIS DERIVADAS
    'VDF004': "Faixa de rendimento domiciliar per capita",

    # Módulo A - Informações do Domicílio
    'A001': "Tipo do domicílio",
    'A02201': "Em seu domicílio, há algum animal de estimação?",

    # Módulo B - Visitas domiciliares de Equipe de Saúde da Família e Agentes de Endemias
    'B001': "O seu domicílio está cadastrado na unidade de saúde da família",

    # Módulo C - Características gerais dos moradores
    'C001': "Número de pessoas no domicílio",
    'C006': "Sexo",
    'C008': "Idade do morador na data de referência",
    'C009': "Cor ou raça",
    'C01001': "Cônjuge ou companheiro(a) mora em nesse domicílio.",

    # Módulo D - Características de educação dos moradores
    'D001': "Sabe ler e escrever?",
    'D00901': "Qual foi o curso mais elevado?",

}

df_all1 = selected[list(PNS_SELECTED_VAR.keys()) + VIOLENCE_KEYS]
df_all2 = selected.filter(regex='^[A]')
#df_all2 = selected.filter(regex='^[C|Q|P]')

df_all = pd.concat([df_all1, df_all2], axis=1)

df_all

In [47]:
PNS_SELECTED_VAR = {
    # Parte 1 - Identificação e Controle
    'V0001': "Unidade da Federação",
    'V0022': "Total de moradores",
    'V0026': "Tipo de situação censitária",
    'V0031': "Tipo de área",
    'V0025A': "Seleção do morador de 15 anos ou mais para responder o questionário individual",


    # VARIÁVEIS DERIVADAS
    'VDF004': "Faixa de rendimento domiciliar per capita",

    # Módulo A - Informações do Domicílio
    'A001': "Tipo do domicílio",
    'A02201': "Em seu domicílio, há algum animal de estimação?",

    # Módulo B - Visitas domiciliares de Equipe de Saúde da Família e Agentes de Endemias
    'B001': "O seu domicílio está cadastrado na unidade de saúde da família",

    # Módulo C - Características gerais dos moradores
    'C001': "Número de pessoas no domicílio",
    'C006': "Sexo",
    'C008': "Idade do morador na data de referência",
    'C009': "Cor ou raça",
    'C01001': "Cônjuge ou companheiro(a) mora em nesse domicílio.",


    # Módulo D - Características de educação dos moradores
    'D001': "Sabe ler e escrever?",
    'D00901': "Qual foi o curso mais elevado?",

    # Módulo P - Estilos de vida
    'P00104': "Peso (em kg)",
    'P00404': "Altura (em cm)",
    'P006': "Em quantos dias da semana o(a) Sr(a) costuma comer feijão? ",
    'P00901': "Em quantos dias da semana, o(a) Sr(a) costuma comer pelo menos um tipo de verdura ou legume (sem contar batata, mandioca, cará ou inhame) como alface, tomate, couve, cenoura, chuchu, berinjela, abobrinha? ",
    'P01101': "Em quantos dias da semana o(a) Sr(a) costuma comer carne vermelha (boi, porco, cabrito, bode, ovelha etc.)? ",
    'P013': "Em quantos dias da semana o(a) Sr(a) costuma comer frango/galinha? ",
    'P015': "Em quantos dias da semana o(a) Sr(a) costuma comer peixe?  ",
    'P018': "Em quantos dias da semana o(a) Sr(a) costuma comer frutas? ",
    'P019': "Em geral, quantas vezes por dia o(a) Sr(a) come frutas?  ",
    'P023': "Em quantos dias da semana o(a) Sr(a) costuma tomar leite? (de origem animal: vaca, cabra, búfala etc.)?",

}

VIOLENCE_KEYS = ['V001', 'V00101', 'V00201', 'V00202', 'V00203', 'V00204', 'V00205', 'V003', 'V006', 'V007', 'V01401', 'V01402', 'V01403', 'V01404', 'V01405', 'V015', 'V018', 'V019', 'V02701', 'V02702', 'V02801', 'V02802', 'V029', 'V032', 'V033', 'V034', 'V03501', 'V03502', 'V03503', 'V036', 'V037', 'V038', 'V039']

SUBJECT_KEYS = ['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS', 'V0026']

df_all1 = selected[list(PNS_SELECTED_VAR.keys()) + VIOLENCE_KEYS]
df_all2 = selected.filter(regex='^[PQ]')
#df_all2 = selected.filter(regex='^[C|Q|P]')

df_all = pd.concat([df_all1, df_all2], axis=1)

In [65]:
df_all

,V0001,V0022,V0026,V0031,V0025A,VDF004,A001,A02201,B001,C001,...,Q12501,Q12607,Q12608,Q12609,Q12610,Q127,Q128,Q132,Q133,Q134
0,11,6.0,1,1,1.0,2.0,1.0,1.0,1.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
9,11,4.0,1,1,1.0,2.0,1.0,1.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
10,11,8.0,1,1,1.0,3.0,1.0,1.0,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
18,11,1.0,1,1,1.0,2.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
19,11,2.0,1,1,1.0,1.0,2.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293709,53,3.0,2,1,1.0,4.0,1.0,1.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
293712,53,2.0,2,1,1.0,4.0,1.0,1.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
293713,53,4.0,2,1,1.0,2.0,1.0,1.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
293718,53,3.0,2,1,1.0,5.0,1.0,1.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN


In [68]:
n_nan_cols = (df_all.isna().mean() <= 0.2).sum()

print(n_nan_cols)  # imprime 2

123


In [66]:
df_all.to_csv('../data/processed/PNS_2019/pns_filtered.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))


In [60]:
df_all.dropna().mean() >= 0.8

V0001     False
V0022     False
V0026     False
V0031     False
V0025A    False
          ...  
Q127      False
Q128      False
Q132      False
Q133      False
Q134      False
Length: 441, dtype: bool

In [61]:
df_all.dropna().mean()

V0001     NaN
V0022     NaN
V0026     NaN
V0031     NaN
V0025A    NaN
         ... 
Q127      NaN
Q128      NaN
Q132      NaN
Q133      NaN
Q134      NaN
Length: 441, dtype: object

In [59]:
# filtrar o DataFrame pelas colunas que possuem pelo menos 80% dos dados não nulos
filtered_df = df_all.loc[:, (df_all.dropna().mean() >= 0.8)]

filtered_df

""
0
9
10
18
19
...
293709
293712
293713
293718


In [38]:
with open('test', 'r') as file:
    lines = file.readlines()
    string_list = []
    for line in lines:
        if not line.isspace() and line.strip():
            string_list.append(line.strip())
print(string_list)

['V001', 'V00101', 'V00201', 'V00202', 'V00203', 'V00204', 'V00205', 'V003', 'V006', 'V007', 'V01401', 'V01402', 'V01403', 'V01404', 'V01405', 'V015', 'V018', 'V019', 'V02701', 'V02702', 'V02801', 'V02802', 'V029', 'V032', 'V033', 'V034', 'V03501', 'V03502', 'V03503', 'V036', 'V037', 'V038', 'V039']


In [41]:
df_all.dropna(axis=1).columns

Index(['V0001', 'V0031', 'A001', 'A02201', 'C001', 'B001', 'C006', 'C008',
       'C009', 'C01001', 'D001', 'P00102', 'P00201', 'P00402', 'P00601',
       'P00602', 'P00603', 'P00604', 'P00605', 'P00607', 'P00608', 'P00609',
       'P00610', 'P00611', 'P00612', 'P00613', 'P00614', 'P00615', 'P00616',
       'P00617', 'P00618', 'P00619', 'P00620', 'P00621', 'P00622', 'P00623',
       'P006', 'P00901', 'P01101', 'P013', 'P015', 'P02001', 'P01601', 'P018',
       'P02002', 'P023', 'P02501', 'P02602', 'P02601', 'P027', 'P034', 'P042',
       'P044', 'P04501', 'P04502', 'P046', 'P04701', 'P050', 'P067', 'P06701',
       'P068', 'P069', 'P06901', 'P07004', 'P07005', 'P07006', 'P07007',
       'P07101', 'Q00101', 'Q02901', 'Q05901', 'Q06306', 'Q068', 'Q074',
       'Q079', 'Q084', 'Q088', 'Q092', 'Q11006', 'Q11604', 'Q120', 'Q124',
       'Q128', 'Q132'],
      dtype='object')

In [ ]:

df_all = selected[SUBJECT_KEYS]
selected = df_all
df_all

Vamos embaralhar estes dados e salva-los na pasta processed/PNS/all.zip

In [5]:
df_all = selected.sample(frac=1, random_state=42).reset_index(drop=True)
df_all.to_csv('../data/processed/PNS_2019/all.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))

Dividindo as base em treino e teste

In [6]:
df_train, df_test = train_test_split(df_all, test_size=0.5, random_state=42)
df_train.to_csv('../data/processed/PNS_2019/train.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))
df_test.to_csv('../data/processed/PNS_2019/test.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))

Com isso temos que as pessoas entrevistadas moram em casas diferentes, isso facilita a divisão dos dados.